### $Q^0 = 1$
### $Z^1 = A Q $
### $Z^1 -> (Q^1, R^1)$

In [2]:
import numpy as np


def forsub(L,bs): #### Metemos 2 matrices, la diagonal inferior de L y la matriz de resultados
    n = bs.size   #### Definimos a n, el cual es el tamaño de nuestra fila  de resultados 
    xs = np.zeros(n) ### Creamos, la matriz donde vamos a guardar nuestros resultados
    for i in range(n):  ####  Usamos la ecuación  4.55
        #### Para encontrar los valores de nuestras x´s
        xs[i] = (bs[i] - L[i,:i]@xs[:i])/L[i,i] 
    #print(xs) 
    return xs


def backsub(U,bs): ##### Definimos nuestra matriz superior y nuestra columna de resultados
    n = bs.size  ### definimos un valor el cual es el tamaño de nuestra matriz de soluciones
    xs = np.zeros(n) #### Definimos  una nueva matriz en la cual colocaremos nuestras soluciones
    
    for i in reversed(range(n)): ### Usando la ecuación 4.60
        #### Obtenemos las soluciones, pero, usando una regresión
        xs[i] = (bs[i] - U[i,i+1:]@xs[i+1:])/U[i,i]

    return xs

def testcreate(n,val):
    #### Vamos a crear las matrices, de tamaño nxn, la cual estará creada de manera escalonada
    ### Y luego sacaremos su raiz
    A = np.arange(val,val+n*n).reshape(n,n)
    A = np.sqrt(A)
    #### De la matriz A obtenemos la matriz de resultados, donde sus elementos se obtienen elevando a la 2.3 
    #### los elementos de la primera fila de A
    bs = (A[0,:])**2.1
    #print(bs)
    return A, bs


def testsolve(f,A,bs):
    xs = f(A,bs); print(xs) ### Vamos a imprimir los resultados usando nuestras funciones 
    xs = np.linalg.solve(A,bs); print(xs) ### Usando la libreria de linalg
    
    

A, bs = testcreate(4,21)
L = np.tril(A) #### Extrae la parte superior 
#### Aplicamos el test, usando la función forsub, y la matriz superior con los resultados bs
testsolve(forsub,L,bs)
print(" ")
U = np.triu(A) 
#### #### Aplicamos el test, usando la función backsub y la matriz inferior con los resultados bs
testsolve(backsub,U,bs)

[ 5.33605887 -0.19676761 -0.13541854 -0.09524368]
[ 5.33605887 -0.19676761 -0.13541854 -0.09524368]
 
[0.14941285 0.10032435 0.06814924 4.6888955 ]
[0.14941285 0.10032435 0.06814924 4.6888955 ]


In [3]:
def mag(xs):
    ### Obtenemos la normalización de cualquier matriz columna o fila
    return np.sqrt(np.sum(xs*xs))

def power(A,kmax=6): #### Igresamos la matriz a la queremos obtener sus
    ### Eigen valores, y k mak, las iteraciones para este metodo de potencias
    zs = np.ones(A.shape[0]) ### creamos un vector con el tamaño del número
    ### de filas  de A
    qs = zs/mag(zs) ### Normalizamos el vector zs
    for k in range(1,kmax):
        zs = A@qs #### Multiplica la matriz A por el vector normalizado
        qs = zs/mag(zs) #### reasinamos el nuevo valor qs, con el resultado anteior
        #print(k,qs)
    lam = qs@A@qs ### Obtenemos los eigenvalores 
    return lam, qs #### Obtenemos su eigenvector y eigenvalor asociado 

def testeigone(f,A,indx=0):
    eigval, eigvec = f(A) ### Aqui testeamos el metodo, usando
    ### La librería  dedicada y nuestro progrma 
    print(" "); print(eigval); print(eigvec)
    npeigvals, npeigvecs = np.linalg.eig(A)
    print(" ")
    print(npeigvals[indx]); print(npeigvecs[:,indx])
    
A,_ = testcreate(4,21)
testeigone(power,A)


 
21.316662663452007
[0.44439562 0.48218122 0.51720906 0.55000781]
 
21.316662663452043
[0.44439562 0.48218122 0.51720906 0.55000781]


In [4]:
def qrdec(A):
    n = A.shape[0] ### Obtemos el número de filas de la matriz A
    Ap = np.copy(A) #### Copiamos los valores de la matriz en Ap
    Q = np.zeros((n,n)) #### Creamos un arreglo nxn de ceros para guardar nuestra matriz Q
    R = np.zeros((n,n)) ### Hacemos lo mismo para R 
    for j in range(n):
        for i in range(j): ### Aplicamos el metodo QR 
            R[i,j] = Q[:,i]@A[:,j] 
            Ap[:,j] -= R[i,j]*Q[:,i] #### De la definición iteramos hasta obtener todos los valores de Ap
        R[j,j] = mag(Ap[:,j]) ### Reasignamos un nuevo valor de R y con ello a Q
        Q[:,j] = Ap[:,j]/R[j,j]
    return Q, R

def testqrdec(A):
    n = A.shape[0] ### Obtemos el número de filas de la matriz A
    Q, R = qrdec(A) #### Aplicamos el metodo qr
    diffa = A - Q@R #### Obtenemos la diferencia para ver que tan parecida es nuestra qr a A
    diffq = np.transpose(Q)@Q - np.identity(n) ### Hacemos lo mismo para nuestras q´s
    print(n, mag(diffa), mag(diffq))

    
for n in range(4,20,2):
    A,_ = testcreate(n,21)
    testqrdec(A)

4 0.0 0.00757477938982088
6 2.5121479338940403e-15 2.446997224625433
8 4.5288390936029406e-15 3.508585247394872
10 8.379058453350832e-15 5.7407941575740855
12 9.81025129326275e-15 8.375274941271778
14 1.656875357463295e-14 10.234951045392204
16 2.398081733190338e-14 12.44307749705754
18 3.4616177071287305e-14 14.42397726224636


In [11]:
def compute_H(nmax, L, omega):
    m = 1 #9.1093837015e-31
    hbar = 1 # 1.055e-34
    mu = np.sqrt(hbar/m*omega)
    
    H = np.zeros((2*nmax + 1, 2*nmax + 1))
    n_x = H.shape[0]
    
    for i in range(n_x): 
        print(H)
        for j in range(n_x):
            if i == j:
                k_n = (2*hbar**2 * np.pi**2 * i**2) / (m* L**2 )
                H[i,j] = k_n +  ( m * omega ** 2 * L**2) / 24
            else:
                integral = ((-1)**(i - j) * (L**2* m* omega**2  ) )/ (4* np.pi**2  * (i - j)**2  )
                H[i, j] =   integral
                H[j, i] =   integral
    return H

L = np.sqrt(12) # 10e-10  
omega = 1 #7.81e20 
nmax = 3
H = compute_H(nmax, L, omega)


def qrmet(inA,kmax=100):
    A = np.copy(inA)
    for k in range(1,kmax):
        Q, R = qrdec(A)
        A = R@Q
    print(k, np.diag(A))
    qreigvals = np.diag(A)
    return qreigvals

M = qrmet(H)
M

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
[[ 0.5        -0.30396355  0.07599089 -0.03377373  0.01899772 -0.01215854
   0.00844343]
 [-0.30396355  0.          0.          0.          0.          0.
   0.        ]
 [ 0.07599089  0.          0.          0.          0.          0.
   0.        ]
 [-0.03377373  0.          0.          0.          0.          0.
   0.        ]
 [ 0.01899772  0.          0.          0.          0.          0.
   0.        ]
 [-0.01215854  0.          0.          0.          0.          0.
   0.        ]
 [ 0.00844343  0.          0.          0.          0.          0.
   0.        ]]
[[ 0.5        -0.30396355  0.07599089 -0.03377373  0.01899772 -0.01215854
   0.00844343]
 [-0.30396355  2.14493407 -0.30396355  0.07599089 -0.03377373  0.01899772
  -0.01215854]
 [ 0.07599089 -0.30396355  0.          0.          0.          0.
   0.      

array([59.72297164, 41.62473778, 26.82091334, 15.30791883,  7.08789843,
        2.17903245,  0.44552762])

In [8]:
np.linalg.eigvals(H)

array([65.06349422, 45.32124082, 29.17142481,  0.4159329 ,  2.27953587,
        7.64249929, 16.61129642])